In [2]:
import pandas as pd
df = pd.read_csv('ranking_comentarios.csv')

In [10]:
df.shape

(11143, 2)

In [4]:
df = df[['notes', 'doc_eval']]

In [5]:
df['doc_eval'].value_counts()

5    68985
0    38945
4     3109
1     2817
3      888
2      312
-        1
Name: doc_eval, dtype: int64

In [6]:
df = df[df['doc_eval'] != "0"]

In [7]:
df = df[df['doc_eval'] != "-"]

In [8]:
df['doc_eval'] = df['doc_eval'].replace(["1","2","3"], "0")

In [9]:
df = df.groupby('doc_eval', group_keys=False).apply(lambda x: x.sample(min(len(x), 4017)))

In [11]:
df.index = range(11143)

In [12]:
df['doc_eval'] = pd.to_numeric(df['doc_eval'])

In [13]:
X = df.notes.to_list()
y = df.doc_eval.to_list()

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [15]:
y_encoder = le.transform(y)
y = le.transform(y)


In [16]:
len(X)

11143

In [17]:
len(y)

11143

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)


In [20]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 7.2MB/s 
     |████████████████████████████████| 890kB 11.6MB/s 
     |████████████████████████████████| 3.0MB 29.8MB/s 
     |████████████████████████████████| 1.2MB 39.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b72966c88ad8532bd548f1304c91124f488be4ed93bceec5809aaba3ccac7a44
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [21]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

#import tensorflow as tf
import json

In [22]:
training_sentences = X_train
validation_sentences = X_test
training_labels =   y_train
validation_labels = y_test

In [23]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')

In [24]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [25]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [26]:
df.doc_eval.nunique()

3

In [27]:
# We classify two labels in this example. In case of multiclass classification, adjust num_labels value
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased',
                                                              num_labels=3)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream ta

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(32),
          epochs=1,
          batch_size=32,
          validation_data=val_dataset.shuffle(100).batch(32))

191/314 [=================>............] - ETA: 57:59 - loss: 0.7857 - accuracy: 0.6288

In [ ]:
test_sentence = "regular"

# replace to test_sentence_sarcasm variable, if you want to test sarcasm
predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = model.predict(predict_input)[0]
print(tf_output)

[[ 0.9571184   0.39767262 -1.2159972 ]]


In [ ]:
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)


[0.59334916 0.33911434 0.06753644]


In [ ]:
import numpy as np
le.inverse_transform([np.argmax(tf_prediction)])

array([0])